In [ ]:
# 1. Install required packages
%pip install pandas ipython-sql prettytable
import prettytable
prettytable.DEFAULT = 'DEFAULT'

In [ ]:
# 2. Import required libraries
import sqlite3, csv

%load_ext sql


In [ ]:
# 3. Create database connection
con = sqlite3.connect("/Users/art/Public/chicago_socioeconomic/FinalDB.db")
cur = con.cursor()

In [ ]:
# 4. Enable SQL magic
%sql sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db

In [ ]:
# create 3 dataframes one for each dataset
#df1 census data
import pandas

df1 = pandas.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')
df1.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")


In [ ]:
#df 2  chigago public schools data
df2 = pandas.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')
df2.to_sql("CHICAGO_PUBLIC_SCHOOLS", con, if_exists='replace', index=False,method="multi")

In [ ]:
# chicago crime data
df3 = pandas.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')
df3.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False,method="multi")

In [ ]:
# make sure the table got created
%sql SELECT name FROM sqlite_master WHERE type='table';

In [ ]:
#Find the total number of crimes recorded in the CRIME table
%sql SELECT count (CASE_NUMBER) from CHICAGO_CRIME_DATA; #533

In [ ]:
#List community area names and numbers with per capita income less than 11000
%sql SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME<11000;

In [ ]:
#List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)
%sql SELECT `CASE_NUMBER` FROM CHICAGO_CRIME_DATA WHERE `DESCRIPTION` LIKE '%minor%';

In [ ]:
#List all kidnapping crimes involving a child?
%sql SELECT * FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE='KIDNAPPING' AND DESCRIPTION LIKE '%child%'

In [ ]:
#List the kind of crimes that were recorded at schools.  (No repetitions)
%sql SELECT distinct (primary_type) FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION like 'SCHOOL%';

In [ ]:
#List the type of schools along with the average safety score for each type.
%sql SELECT `Elementary, Middle, or High School`, AVG(SAFETY_SCORE) from CHICAGO_PUBLIC_SCHOOLS group by 1;

In [ ]:
#List 5 community areas with highest % of households below poverty line
%sql SELECT COMMUNITY_AREA_NUMBER, PERCENT_HOUSEHOLDS_BELOW_POVERTY from CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5;

In [ ]:
#Which community area is most crime prone? Display the coumminty area number only.

%sql SELECT  c.COMMUNITY_AREA_NUMBER, count (cr.id) from  CENSUS_DATA c, CHICAGO_CRIME_DATA cr where cr.COMMUNITY_AREA_NUMBER = c.COMMUNITY_AREA_NUMBER group by 1 order by 2 desc;



In [ ]:
#Use a sub-query to find the name of the community area with highest hardship index
%sql select COMMUNITY_AREA_NAME from CENSUS_DATA where HARDSHIP_INDEX in (select max(HARDSHIP_INDEX) from CENSUS_DATA);



In [ ]:
#Use a sub-query to determine the Community Area Name with most number of crimes?

%sql select COMMUNITY_AREA_NAME from CENSUS_DATA where COMMUNITY_AREA_NUMBER = (select COMMUNITY_AREA_NUMBER from CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY count (*) DESC);